In [ ]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

In [ ]:
level = 2
#setup = 'oversigma_10tissue'
#label = 'disease_type'
label='primary_site'
#label = 'uniq'
L = 4
#labels = ['primary_site', 'disease_type']
#labels = ['primary_site', 'secondary_site']
#directory = "results/hSBM/%s"%setup
#directory="gtex/hsbm/%s"%setup
directory="merged/hsbm"
df = pd.read_csv("%s/mainTable.csv"%(directory), index_col=[0], header=[0])
df.head()

In [ ]:
data=df.T.values[:901]

In [ ]:
data.shape

In [ ]:
dend = shc.dendrogram(shc.linkage(data, method='ward'))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')  
out = cluster.fit_predict(data)

In [ ]:
from hsbmpy import define_labels, get_cluster_given_l

In [ ]:
true_labels, predicted_labels = define_labels(get_cluster_given_l(2,directory),pd.read_csv("%s/files.dat"%directory, index_col=[0]), label='primary_site')

In [ ]:
np.shape(out)

In [ ]:
np.shape(predicted_labels)

In [ ]:
np.shape(true_labels)

In [ ]:
from sklearn.metrics import homogeneity_completeness_v_measure

In [ ]:
homogeneity_completeness_v_measure(true_labels,predicted_labels)

In [ ]:
df_files = pd.read_csv("%s/files.dat"%directory, index_col=[0])
df_files.head()

In [ ]:
from hsbmpy import get_file

In [ ]:
xl=[901, 199, 30, 9, 1]

In [ ]:
scores = {}
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}

In [ ]:
for x in xl:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(data)
    true_out = []
    for sample in df.columns.values[0:901]:
        try:
            true_out.append(get_file(sample, df_files)['primary_site'])
        except:
            true_out.append('')
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierarchical']['h'].append(score[0])
    scores['hierarchical']['c'].append(score[1])
    scores['hierarchical']['V'].append(score[2])

In [ ]:
scores['hierarchical']

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=100,random_state=42)
topics = lda.fit_transform(data)

In [ ]:
cm = sns.clustermap(topics)

In [ ]:
cluster.fit_predict(topics)

In [ ]:
scores['lda']={
    'h':[],
    'c':[],
    'V':[]
}
for x in xl:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(topics)
    true_out = []
    for sample in df.columns.values[0:901]:
        try:
            true_out.append(get_file(sample, df_files)['primary_site'])
        except:
            true_out.append('')
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['lda']['h'].append(score[0])
    scores['lda']['c'].append(score[1])
    scores['lda']['V'].append(score[2])

In [ ]:
scores['lda']

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
df = pd.read_csv("mainTable.csv", index_col=[0], header=[0])

In [ ]:
tfdf = tf.convert_to_tensor(df.transpose().values, dtype=tf.float64)
tfdf.shape

In [ ]:
def euclidean_self(input_tensor):
    tensor_shape = input_tensor.get_shape().as_list()
    tensor_iter = input_tensor
    euclidean_list = []

    ses = tf.Session()

    for i in range(tensor_shape[0]):
        if i%100==0:
            print(i)
        split_head, split_tail = tf.split(tensor_iter, [1, tensor_shape[0]-1])
        tensor_iter = tf.concat([split_tail, split_head], 0)
        euclidean_dist = (tf.sqrt(tf.reduce_sum(tf.square(input_tensor-tensor_iter), 1)))
        ses.run(euclidean_dist)
        euclidean_row = euclidean_dist.eval(session=ses)
        euclidean_list.append(euclidean_row)

    ses.close()

    euclidean_out = np.asarray(euclidean_list).transpose([1, 0])

    for i in range(tensor_shape[0]):
        euclidean_out[i] = np.append(euclidean_out[i][tensor_shape[0]-1-i:], 
                                     euclidean_out[i][:tensor_shape[0]-1-i])

    '''
    the return type is numpy.array, if need tensor for return, use the code beneath
    '''
    # euclidean_out = tf.convert_to_tensor(euclidean_out)

    return euclidean_out

In [ ]:
euclidean_self(tfdf)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(alternate))

In [ ]:
df.head()

In [ ]:
tfdf[0].shape